In [3]:
import numpy as np
import pandas as pd
import re 
from tqdm import tqdm

In [4]:
pd.set_option('display.max_columns', None)  # Show all columns
pd.set_option('display.max_rows', None)    # Show all rows
pd.set_option('display.max_colwidth', None) # Allow wrapping for long values

In [5]:
regex = '^(?P<client>\S+) \S+ (?P<userid>\S+) \[(?P<datetime>[^\]]+)\] "(?P<method>[A-Z]+) (?P<request>[^ "]+)? HTTP/[0-9.]+" (?P<status>[0-9]{3}) (?P<size>[0-9]+|-) "(?P<referrer>[^"]*)" "(?P<useragent>[^"]*)'
columns = ['client', 'userid', 'datetime', 'method', 'request', 'status', 'size', 'referer', 'user_agent']

In [6]:
def logs_to_df(logfile, output_dir, errors_file):
    """
    Parse log file and save parsed data into DataFrame, then save DataFrame to Parquet files.

    Args:
        logfile (str): Path to the log file.
        output_dir (str): Directory to save Parquet files.
        errors_file (str): Path to the file to store parsing errors.

    Returns:
        None
    """
    with open(logfile) as source_file:
        linenumber = 0
        parsed_lines = []
        for line in tqdm(source_file):
            try:
                log_line = re.findall(regex, line)[0]
                parsed_lines.append(log_line)
            except Exception as e:
                with open(errors_file, 'at') as errfile:
                    print((line, str(e)), file=errfile)
                continue
            linenumber += 1
            if linenumber % 250_000 == 0:
                df = pd.DataFrame(parsed_lines, columns=columns)
                df.to_parquet(f'{output_dir}/file_{linenumber}.parquet')
                parsed_lines.clear()
        else:
            df = pd.DataFrame(parsed_lines, columns=columns)
            df.to_parquet(f'{output_dir}/file_{linenumber}.parquet')
            parsed_lines.clear()
            

In [7]:
logs_to_df(logfile='/home/mehmet/Desktop/linkedin/archive/access.log', output_dir="./outputs" , errors_file='errors.txt')

10365152it [00:47, 220278.92it/s]


In [8]:
logs_df = pd.read_parquet('./outputs')

In [9]:
logs_df.shape

(10364865, 9)

In [10]:
logs_df.head()

client userid                    datetime method  \
0  37.152.163.59      -  22/Jan/2019:12:38:27 +0330    GET   
1  37.152.163.59      -  22/Jan/2019:12:38:27 +0330    GET   
2    85.9.73.119      -  22/Jan/2019:12:38:27 +0330    GET   
3  37.152.163.59      -  22/Jan/2019:12:38:27 +0330    GET   
4    85.9.73.119      -  22/Jan/2019:12:38:27 +0330    GET   

                                                     request status  size  \
0  /image/29314?name=%D8%AF%DB%8C%D8%A8%D8%A7-7.jpg&wh=50x50    200  1105   
1                           /static/images/zanbil-kharid.png    200   358   
2                                    /static/images/next.png    200  3045   
3  /image/29314?name=%D8%AF%DB%8C%D8%A8%D8%A7-4.jpg&wh=50x50    200  1457   
4                                 /static/images/checked.png    200  1083   

                                                                                                                                                                                                            referer  \
0  https://www.zanbil.ir/product/29314/%DA%A9%D8%A7%D9%84%D8%B3%DA%A9%D9%87-%D8%AF%D9%88%D9%82%D9%84%D9%88-%D8%AF%D9%84%DB%8C%D8%AC%D8%A7%D9%86-%D9%85%D8%AF%D9%84-%D8%AF%DB%8C%D8%A8%D8%A7-Delijan-Twin-Strollers-   
1  https://www.zanbil.ir/product/29314/%DA%A9%D8%A7%D9%84%D8%B3%DA%A9%D9%87-%D8%AF%D9%88%D9%82%D9%84%D9%88-%D8%AF%D9%84%DB%8C%D8%AC%D8%A7%D9%86-%D9%85%D8%AF%D9%84-%D8%AF%DB%8C%D8%A8%D8%A7-Delijan-Twin-Strollers-   
2                                                                                                                                                                https://znbl.ir/static/bundle-bundle_site_head.css   
3  https://www.zanbil.ir/product/29314/%DA%A9%D8%A7%D9%84%D8%B3%DA%A9%D9%87-%D8%AF%D9%88%D9%82%D9%84%D9%88-%D8%AF%D9%84%DB%8C%D8%AC%D8%A7%D9%86-%D9%85%D8%AF%D9%84-%D8%AF%DB%8C%D8%A8%D8%A7-Delijan-Twin-Strollers-   
4                                                                                                                                                                https://znbl.ir/static/bundle-bundle_site_head.css   

                                                                                                          user_agent  
0                                               Mozilla/5.0 (Windows NT 6.1; WOW64; Trident/7.0; rv:11.0) like Gecko  
1                                               Mozilla/5.0 (Windows NT 6.1; WOW64; Trident/7.0; rv:11.0) like Gecko  
2  Mozilla/5.0 (Windows NT 6.1; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/71.0.3578.98 Safari/537.36  
3                                               Mozilla/5.0 (Windows NT 6.1; WOW64; Trident/7.0; rv:11.0) like Gecko  
4  Mozilla/5.0 (Windows NT 6.1; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/71.0.3578.98 Safari/537.36

In [11]:
logs_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10364865 entries, 0 to 10364864
Data columns (total 9 columns):
 #   Column      Dtype 
---  ------      ----- 
 0   client      object
 1   userid      object
 2   datetime    object
 3   method      object
 4   request     object
 5   status      object
 6   size        object
 7   referer     object
 8   user_agent  object
dtypes: object(9)
memory usage: 711.7+ MB


In [12]:
logs_df.describe()

client    userid                    datetime    method  \
count        10364865  10364865                    10364865  10364865   
unique         258445         2                      400107         6   
top     66.249.66.194         -  26/Jan/2019:19:07:39 +0330       GET   
freq           353483  10364863                         368  10189772   

               request    status      size   referer  \
count         10364865  10364865  10364865  10364865   
unique          893045        15     69707    103238   
top     /settings/logo       200         0         -   
freq            352047   9579824    770606   1646072   

                                                                                                                user_agent  
count                                                                                                             10364865  
unique                                                                                                               28339  
top     Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/71.0.3578.98 Safari/537.36  
freq                                                                                                                746572

In [13]:
logs_df.isnull().sum()

client        0
userid        0
datetime      0
method        0
request       0
status        0
size          0
referer       0
user_agent    0
dtype: int64

In [14]:
logs_df.nunique()

client        258445
userid             2
datetime      400107
method             6
request       893045
status            15
size           69707
referer       103238
user_agent     28339
dtype: int64

In [15]:
print(logs_df["userid"].unique())

logs_df.drop(columns = ["userid"], axis = 0, inplace=True)

['-' 'admin']


In [16]:
if logs_df.duplicated().sum():
    logs_df = logs_df.drop_duplicates()
    
logs_df.nunique()

client        258445
datetime      400107
method             6
request       893045
status            15
size           69707
referer       103238
user_agent     28339
dtype: int64

In [17]:
df_samp = logs_df.copy()

In [18]:
#veriler hangi aylardan?
months_list = df_samp["datetime"].apply(lambda x: x.split("/")[1])
print(months_list.unique())

['Jan']


In [19]:
#datetime string dönüşümü
df_samp['datetime'] = df_samp['datetime'].apply(lambda x: x.split('+')[0][:-1]) #remove the extra part i.e. + 3:30 from the datetime column
df_samp['datetime'] = df_samp.datetime.apply(lambda x: x.replace('Jan', '01')) #since the data given is only for jan
df_samp['datetime'] = pd.to_datetime(df_samp.datetime, format='%d/%m/%Y:%H:%M:%S')

In [20]:
#datetime -> day, hour, minute dönüşümü
df_samp['day'] = df_samp['datetime'].apply(lambda x: x.day)
df_samp['hour'] = df_samp['datetime'].apply(lambda x: x.hour)
df_samp['minute'] = df_samp['datetime'].apply(lambda x: x.minute)

In [21]:
df_samp.head(1)

client            datetime method  \
0  37.152.163.59 2019-01-22 12:38:27    GET   

                                                     request status  size  \
0  /image/29314?name=%D8%AF%DB%8C%D8%A8%D8%A7-7.jpg&wh=50x50    200  1105   

                                                                                                                                                                                                            referer  \
0  https://www.zanbil.ir/product/29314/%DA%A9%D8%A7%D9%84%D8%B3%DA%A9%D9%87-%D8%AF%D9%88%D9%82%D9%84%D9%88-%D8%AF%D9%84%DB%8C%D8%AC%D8%A7%D9%86-%D9%85%D8%AF%D9%84-%D8%AF%DB%8C%D8%A8%D8%A7-Delijan-Twin-Strollers-   

                                                             user_agent  day  \
0  Mozilla/5.0 (Windows NT 6.1; WOW64; Trident/7.0; rv:11.0) like Gecko   22   

   hour  minute  
0    12      38

In [22]:
from urllib.parse import urlparse

#sağlayıcı bilgisinin eklenmesi
def extract_domain(url):
    parsed_url = urlparse(url)
    return parsed_url.netloc

In [23]:
df_samp["referers"] = df_samp["referer"].apply(extract_domain)

In [24]:
df_samp.info()

<class 'pandas.core.frame.DataFrame'>
Index: 10256742 entries, 0 to 10364864
Data columns (total 12 columns):
 #   Column      Dtype         
---  ------      -----         
 0   client      object        
 1   datetime    datetime64[ns]
 2   method      object        
 3   request     object        
 4   status      object        
 5   size        object        
 6   referer     object        
 7   user_agent  object        
 8   day         int64         
 9   hour        int64         
 10  minute      int64         
 11  referers    object        
dtypes: datetime64[ns](1), int64(3), object(8)
memory usage: 1017.3+ MB


In [28]:
browsers = []
oss = []

# Her satır için User-Agent stringini ayrıştırma
for ua in df_samp['user_agent']:
    browser = re.search(r'(Chrome|Safari|Firefox|Opera|MSIE|Trident|Edge)', ua)
    os = re.search(r'(Windows NT|Mac OS X|Android|iPhone OS|Linux)', ua)
    
    browsers.append(browser.group(0) if browser else 'Other')
    oss.append(os.group(0) if os else 'Other')

# Ayrıştırılan bilgileri DataFrame'e ekleme
df_samp['browser'] = browsers
df_samp['os'] = oss

In [30]:
df_samp.head(10)

client            datetime method  \
0  37.152.163.59 2019-01-22 12:38:27    GET   
1  37.152.163.59 2019-01-22 12:38:27    GET   
2    85.9.73.119 2019-01-22 12:38:27    GET   
3  37.152.163.59 2019-01-22 12:38:27    GET   
4    85.9.73.119 2019-01-22 12:38:27    GET   
5  37.152.163.59 2019-01-22 12:38:27    GET   
6  77.245.233.52 2019-01-22 12:38:27    GET   
7  37.27.128.139 2019-01-22 12:38:27    GET   
8  77.245.233.52 2019-01-22 12:38:27    GET   
9  77.245.233.52 2019-01-22 12:38:27    GET   

                                                                                                                                                            request  \
0                                                                                                         /image/29314?name=%D8%AF%DB%8C%D8%A8%D8%A7-7.jpg&wh=50x50   
1                                                                                                                                  /static/images/zanbil-kharid.png   
2                                                                                                                                           /static/images/next.png   
3                                                                                                         /image/29314?name=%D8%AF%DB%8C%D8%A8%D8%A7-4.jpg&wh=50x50   
4                                                                                                                                        /static/images/checked.png   
5                                                                                                                                        /static/images/loading.gif   
6                                                                                                                                  /image/11082/productType/240x180   
7  /browse/Tablet-Arm-Chair/%D8%B5%D9%86%D8%AF%D9%84%DB%8C-%DA%86%D9%87%D8%A7%D8%B1%D9%BE%D8%A7%DB%8C%D9%87-%D8%AF%D8%A7%D9%86%D8%B4-%D8%A2%D9%85%D9%88%D8%B2%DB%8C   
8                                                                                                                                              /image/851/mainSlide   
9                                                                                                                                              /image/848/mainSlide   

  status   size  \
0    200   1105   
1    200    358   
2    200   3045   
3    200   1457   
4    200   1083   
5    200   7370   
6    200  12458   
7    200  30604   
8    200  89859   
9    200  93168   

                                                                                                                                                                                                            referer  \
0  https://www.zanbil.ir/product/29314/%DA%A9%D8%A7%D9%84%D8%B3%DA%A9%D9%87-%D8%AF%D9%88%D9%82%D9%84%D9%88-%D8%AF%D9%84%DB%8C%D8%AC%D8%A7%D9%86-%D9%85%D8%AF%D9%84-%D8%AF%DB%8C%D8%A8%D8%A7-Delijan-Twin-Strollers-   
1  https://www.zanbil.ir/product/29314/%DA%A9%D8%A7%D9%84%D8%B3%DA%A9%D9%87-%D8%AF%D9%88%D9%82%D9%84%D9%88-%D8%AF%D9%84%DB%8C%D8%AC%D8%A7%D9%86-%D9%85%D8%AF%D9%84-%D8%AF%DB%8C%D8%A8%D8%A7-Delijan-Twin-Strollers-   
2                                                                                                                                                                https://znbl.ir/static/bundle-bundle_site_head.css   
3  https://www.zanbil.ir/product/29314/%DA%A9%D8%A7%D9%84%D8%B3%DA%A9%D9%87-%D8%AF%D9%88%D9%82%D9%84%D9%88-%D8%AF%D9%84%DB%8C%D8%AC%D8%A7%D9%86-%D9%85%D8%AF%D9%84-%D8%AF%DB%8C%D8%A8%D8%A7-Delijan-Twin-Strollers-   
4                                                                                                                                                                https://znbl.ir/static/bundle-bundle_site_head.css   
5  https://www.zanbil.ir/product/29314/%DA%A9%D8%A7%D9%84%D8%B3%DA%A9%D9%87-%D8%AF%D9%88%D9%82%D9%84%D9%88-%D8%AF%D9%84%DB%8C%D8%AC%D8%A7%D9%86-%D9%85%D8%

In [31]:
df_samp.to_csv("df_samp.csv")